<a href="https://colab.research.google.com/github/hyeonseonn/sensing_app/blob/main/0721_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense


In [ ]:
#1. data
np.random.seed(42) # 매번 돌릴때마다 같은 결과 나오도록 seed
days = 30 # 5분 단위 한 달 데이터
points_per_day = 24 * 12
total_points = days * points_per_day

time_index = pd.date_range('2025-06-01', periods=total_points, freq='5T') # T : 분(minutes) 의미

cpu = 50 + 30*np.sin(np.linspace(0, 10, total_points)) + np.random.normal(0, 5, total_points) # 평균 0, 표준편차 5

df = pd.DataFrame({'timestamp': time_index, 'cpu': cpu})
df.set_index('timestamp', inplace=True)

# 2. 정규화
scaler = MinMaxScaler()
scaled_cpu = scaler.fit_transform(df[['cpu']].values)

# 3. sequence 생성 (시계열 -> 예측)
def create_sequences(dataset, past_steps=20, future_steps=5):
    X, y = [], []
    for i in range(len(dataset)-past_steps-future_steps):
        X.append(dataset[i:i+past_steps])
        y.append(dataset[i+past_steps:i+past_steps+future_steps])
    return np.array(X), np.array(y)

past_steps = 20
future_steps = 5
X, y = create_sequences(scaled_cpu, past_steps, future_steps)

# 8:2 학습
train_size = int(len(X)*0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

print(f"입력 X_train shape: {X_train.shape}, 출력 y_train shape: {y_train.shape}")

# 5. CNN-LSTM
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu',
                 input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(64, activation='relu'))
model.add(Dense(future_steps))  # future step에서의 CPU 예측
model.compile(optimizer='adam', loss='mse')


model.fit(X_train, y_train.reshape((y_train.shape[0], -1)), # -1: numpy가 알아서 차원 계산하도록
          epochs=50, batch_size=16, verbose=1)


# 6. prediction
pred = model.predict(X_test)
pred = pred.reshape(-1, future_steps)  # 미래 5분 예측
y_true = y_test.reshape(-1, future_steps)

# visualization
plt.figure(figsize=(12,5))
plt.plot(y_true[:100,0], label="True CPU(1minutes)")
plt.plot(pred[:100,0], label="Estimated CPU(1minutes)")
plt.legend()
plt.title("CNN-LSTM")
plt.show()


입력 X_train shape: (6892, 20, 1), 출력 y_train shape: (6892, 5, 1)
Epoch 1/50


/tmp/ipython-input-8-351977584.py:7: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  time_index = pd.date_range('2025-06-01', periods=total_points, freq='5T') # T : 분(minutes) 의미
/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


431/431 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 0.0427
Epoch 2/50
431/431 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0033
Epoch 3/50
431/431 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0033
Epoch 4/50
431/431 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 0.0033
Epoch 5/50
431/431 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0033
Epoch 6/50
431/431 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0033
Epoch 7/50
431/431 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0033
Epoch 8/50
431/431 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.0033
Epoch 9/50
431/431 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 0.0033
Epoch 10/50
431/431 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0032
Epoch 11/50
431/431 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0032
Epoch 12/50
431/431 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.0033
Epoch 13/50
431/431 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.0033
Epoch 14/50
398/431 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0032